In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('Wimbledon_featured_matches.csv').ffill()

FileNotFoundError: [Errno 2] No such file or directory: 'Wimbledon_featured_matches.csv'

In [ ]:
df['elapsed_time'] = pd.to_timedelta(df['elapsed_time']).apply(lambda x: x.total_seconds() / 60)

In [ ]:
df['p1_place'] = ''
for index, row in df.iterrows():
    # 如果 game_no 为 13，则是决胜局的情况
    if row['game_no'] == 13:
        # 根据局数的差异确定 p1_place 的值
        if row['p1_score'] > row['p2_score']:
            df.at[index, 'p1_place'] = 'ad'  # 球员1处于advantage
        elif row['p1_score'] < row['p2_score']:
            df.at[index, 'p1_place'] = 'da'  # 球员1处于disadvantage
        else:
            df.at[index, 'p1_place'] = 'de'  # 球员1处于deuce
    else:
        # 常规计分方式
        if row['p1_score'] == 'AD':
            df.at[index, 'p1_place'] = 'ad'  # 球员1处于advantage
        elif row['p2_score'] == 'AD':
            df.at[index, 'p1_place'] = 'da'  # 球员1处于disadvantage
        else:
            # 没有AD情况，检查局分是否为 40
            if row['p1_score'] == 40 and row['p2_score'] == 40:
                df.at[index, 'p1_place'] = 'de'  # 球员1处于deuce
            else:
                # 如果都不是，则使用常规计分方式
                if row['p1_score'] > row['p2_score']:
                    df.at[index, 'p1_place'] = 'ad'  # 球员1处于advantage
                elif row['p1_score'] < row['p2_score']:
                    df.at[index, 'p1_place'] = 'da'  # 球员1处于disadvantage
                else:
                    df.at[index, 'p1_place'] = 'de'  # 球员1处于deuce

df.drop(['p1_score', 'p2_score'], axis=1, inplace=True)
df['p1_place'] = df['p1_place'].astype('category').cat.codes
df

In [ ]:
grouped = df.groupby('match_id')
grouped_dfs = {}
for name, group_df in grouped:
    grouped_dfs[name] = group_df

In [ ]:
select1_columns = ['point_victor',
                  'elapsed_time','set_no','game_no',
                  'server','p1_place','p2_points_won','p1_ace','p2_ace','p1_winner','p1_unf_err','p2_unf_err','p2_games','p2_winner',
                  'p1_sets','p1_games','p1_points_won']
train = grouped_dfs['2023-wimbledon-1301'][select1_columns]
test = grouped_dfs['2023-wimbledon-1302'][select1_columns]

train['key_point1']=''
train['key_point1']=train['point_victor'].replace(2,-1).cumsum()


X_train= train.drop(['key_point1','point_victor'],axis=1)
y_train = train['key_point1'].replace(2,0)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

feature_importances = model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

print(feature_importance_df)